# Работа с реляционными базыми данных

### План
1. Оглянемся назад: Реляционная базы данных и Язык SQL
1. SQLite3
1. PostgreSQL + psycopg2
1. ORM. [SQLAlchemy](http://www.sqlalchemy.org)

### Реляционная базы данных

Реляционные базы данных:
  * Связанная информация, хранящаяся в двумерных таблицах
  * Каждая таблица представляет собой набор полей (колонки) и набор объектов (записей)
  * Порядок записей в таблице произволен, столбцов - фиксирован
  * Идентификация записей осуществляется с помощью первичных ключей

Если коротко, то реляционные базы данных – это таблицы с индексами для быстрого поиска, связанные между собой

### Язык SQL

* [Язык структурированных запросов](https://ru.wikipedia.org/wiki/SQL)
* Стандартизирован, но БД пытаются соответствовать стандарту ANSI не позволяя ему ограничивать их слишком сильно
* Типы: Числа (INTEGER и DECIMAL), строки (CHAR, VARCHAR) и т.д.
* Операторы: SELECT FROM, WHERE, GROUP BY, COUNT, HAVING и т.д.
* Джойны: Inner join, outer join и т.д.
* Подзапросы
* Команды модификации: INSERT, DELETE, UPDATE
* Создание таблиц


## [Курсор](https://postgrespro.ru/docs/postgrespro/9.6/plpgsql-cursors)

**Курсор** – специальный объект, выполняющий запрос и получающий его результат.

Вместо того чтобы сразу выполнять весь запрос, есть возможность настроить курсор, инкапсулирующий запрос, и затем получать результат запроса по нескольку строк за раз. Одна из причин так делать заключается в том, чтобы избежать переполнения памяти, когда результат содержит большое количество строк.

В PL/pgSQL:
```sql
name [ [ NO ] SCROLL ] CURSOR [ ( arguments ) ] FOR query;
```


```sql
DECLARE
    curs1 refcursor;
    curs2 CURSOR FOR SELECT * FROM tenk1;
    curs3 CURSOR (key integer) FOR SELECT * FROM tenk1 WHERE unique1 = key;
```

### Открывание курсора


```sql
OPEN unbound_cursorvar [ [ NO ] SCROLL ] FOR query;

OPEN curs1 FOR SELECT * FROM foo WHERE key = mykey;
OPEN curs2;
```

### Открывание курсора на EXECUTE


```sql
OPEN unbound_cursorvar [ [ NO ] SCROLL ] FOR EXECUTE query_string
                                     [ USING expression [, ... ] ];

OPEN curs1 FOR EXECUTE 'SELECT * FROM ' || quote_ident(tabname)
                                        || ' WHERE col1 = $1' USING keyvalue;
```

### Передача параметров в курсор

```sql
DECLARE
    -- Переменная обязательно вводится до курсора
    key integer;
    curs4 CURSOR FOR SELECT * FROM tenk1 WHERE unique1 = key;
    curs3 CURSOR (key integer) FOR SELECT * FROM tenk1 WHERE unique1 = key;
BEGIN
    key := 42;
    OPEN curs4;
    OPEN curs3(42);
```

Если необходимо освободить ресурсы до завершения транзакции или освободить переменную курсора для повторного открывания, можно закрыть уже открытый курсор:

```sql
CLOSE cursor;

CLOSE curs1;
```

## Использование курсоров

Как только курсор открыт, можно начинать работу с данными

## FETCH

FETCH извлекает строку из курсора в заданный таргет.

```sql
FETCH [ direction { FROM | IN } ] cursor INTO target;
```

В качестве direction могут использоваться: `NEXT, FIRST, LAST, ABSOLUTE count, RELATIVE count` и т.д.

```sql
FETCH curs1 INTO rowvar;
FETCH curs2 INTO foo, bar, baz;
FETCH LAST FROM curs3 INTO x, y;
FETCH RELATIVE -2 FROM curs4 INTO x;
```

### [SQLite3](https://docs.python.org/3/library/sqlite3.html)

* Простейший способ использовать БД
* Вся БД хранится в одном файле (или в памяти)
* Отличный способ иметь БД в мобильных приложениях
* Доступ из коммандной строки: sqlite3 filename
* API для Python, с++ и т.д.

#### Попробуем выполнить простой запрос

In [1]:
import sqlite3

# Соединение с базой
conn = sqlite3.connect('example.db')

# Объект для исполнения SQL запросов над базой
cur = conn.cursor()

# Выполним какой-нибудь запрос
cur.execute('select 1')

# Читаем результат. fetchone ожидает единственную строку
print(cur.fetchone())

# Закрываем за собой соединение с базой.
conn.close()

(1,)


Не забываем закрывать соединение! <br>
**Connection - это ресурс, который может и закончиться.** <br>
В этом нам помогут контекстные менеджеры.

In [2]:
# Соединение с базой теперь закроется само.
with sqlite3.connect('example.db') as conn:
    
    # Объект для исполнения SQL запросов над базой
    cur = conn.cursor()

    # Выполним какой-нибудь запрос
    cur.execute("select 'Hello, world!', 6 * 7")

    # Читаем результат. fetchone ожидает единственную строку
    print(cur.fetchone())

('Hello, world!', 42)


Установка соединения может быть дорогой операцией.<br>
Не стоит создавать соединение для каждого запроса. По возможности переиспользуйте старое.

#### Теперь посмотрим как работать с sqlite3 на примере простой базы с пользователями.<br>
Для начала создадим таблицы:

In [3]:
def create_tables(cur, conn):
    # Удалим старые таблицы, если есть.
    cur.execute('DROP TABLE IF EXISTS users')
    cur.execute('DROP TABLE IF EXISTS user_types')
    
    # Пишем SQL запрос для создания таблиц
    cur.execute('''
        CREATE TABLE user_types (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(255)
        )''')
    cur.execute('''
        CREATE TABLE users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            first_name VARCHAR(255),
            last_name VARCHAR(255),
            active BOOLEAN,
            profile INTEGER,
            FOREIGN KEY(profile) REFERENCES user_types(id)
        )''')
    
    # Отправить текущую транзакцию.
    # Если не сделать, то изменения не будут видны в других connect'ax
    # Закрытие соединения не вызывает commit автоматически!
    conn.commit()

Работать с пустыми таблицами не очень интересно, давайте заполним их.

In [4]:
def add_user_types(cur):
    cur.execute('''INSERT INTO user_types (name) VALUES ('Student')''')
    cur.execute('''INSERT INTO user_types (name) VALUES ('Teacher')''')

def add_teachers(cur):
    cur.execute('''
        INSERT INTO users (first_name, last_name, active, profile) VALUES 
            ('Maxim', 'Popov', '1', 
                (SELECT id FROM user_types WHERE name = 'Teacher'))''')
    cur.execute('''
        INSERT INTO users (first_name, last_name, active, profile) VALUES 
            ('Igor', 'Orlov', '1', 
                (SELECT id FROM user_types WHERE name = 'Teacher'))''')

def add_students(cur):
    cur.execute('''
        INSERT INTO users (first_name, last_name, active, profile) VALUES 
            ('Ivan', 'Ivanov', '0', 
                (SELECT id FROM user_types WHERE name = 'Student'))''')
    cur.execute('''
        INSERT INTO users (first_name, last_name, active, profile) VALUES 
            ('Petr', 'Petrov', '0', 
                (SELECT id FROM user_types WHERE name = 'Student'))''')
    cur.execute('''
        INSERT INTO users (first_name, last_name, active, profile) VALUES 
            ('Petr', 'Sidorov', '0', 
                (SELECT id FROM user_types WHERE name = 'Student'))''')

Выполним запросы к базе

In [5]:
with sqlite3.connect('example.db') as conn:

    cur = conn.cursor()
    
    # Функция создания таблиц коммитит транзакцию сразу.
    create_tables(cur, conn)
    
    # Функции добавления польователей не вызывают commit.
    add_user_types(cur)
    add_teachers(cur)
    add_students(cur)
    
    # Делаем commit для всех запросов добавления.
    # Если забудем, то таблицы останутся пустыми.
    conn.commit()

Попробуем теперь зачитать всех студентов

In [6]:
def select_all_students(conn):
    query = '''SELECT users.first_name, users.last_name 
               FROM users 
               WHERE profile = (SELECT id FROM user_types WHERE name = 'Student')'''
    cur = conn.cursor()
    cur.execute(query)

    # Получим все строчки результата выполнения query
    # Это не всегда быстро, особенно если много результатов
    # Кроме того предполагает большое потребление памяти,
    # так как возвращается list
    rows = cur.fetchall() 

    # Как fetchall, только задаем количество получаемых записей
    # rows = cur.fetchmany(10) 

    for row in rows:
        first_name, last_name = row
        print(first_name, last_name)
        
with sqlite3.connect('example.db') as conn:
    select_all_students(conn)

Ivan Ivanov
Petr Petrov
Petr Sidorov


In [7]:
def select_all_students(conn):
    query = '''SELECT users.first_name, users.last_name 
               FROM users 
               WHERE profile = (SELECT id FROM user_types WHERE name = 'Student')'''
    cur = conn.cursor()
    cur.execute(query)

    # А лучше просто проитерироваться
    for row in cur:
        first_name, last_name = row
        print(first_name, last_name)
        
with sqlite3.connect('example.db') as conn:
    select_all_students(conn)

Ivan Ivanov
Petr Petrov
Petr Sidorov


Попробуем сделать учителя Maxim Popov студентом

In [8]:
def alter_teacher(conn):
    cur = conn.cursor()
    cur.execute('''UPDATE users SET profile = (SELECT id FROM user_types WHERE name = "Student")
                   WHERE first_name = "Maxim" AND last_name = "Popov"
    ''')

with sqlite3.connect('example.db') as conn:
    alter_teacher(conn)

Посмотрим на результат

In [9]:
with sqlite3.connect('example.db') as conn:
    select_all_students(conn)

Maxim Popov
Ivan Ivanov
Petr Petrov
Petr Sidorov


Создадим функцию, чтобы добавить всех студентов сразу!

In [10]:
def add_students(conn, students):
    queries = []
    
    for student in students:
        query = '''
            INSERT INTO users (first_name, last_name, active, profile) VALUES 
                ("{}", "{}", 0, 
                    (SELECT id FROM user_types WHERE name = "Student"));'''
        queries.append(query.format(*student))
    
    cur = conn.cursor()
    
    # Исполняет сразу несколько запросов, разделенных ';'
    cur.executescript(''.join(queries))

In [11]:
with sqlite3.connect('example.db') as conn:
    students = [('Vladimir', 'Vladimirov'), ('Dmitry', 'Dmitriev')]
    add_students(conn, students)
    
    # Проверим, что все вставилось.
    select_all_students(conn)

Maxim Popov
Ivan Ivanov
Petr Petrov
Petr Sidorov
Vladimir Vladimirov
Dmitry Dmitriev


Все отлично работает!

In [14]:
with sqlite3.connect('example.db') as conn:
    students = [('cool', 'hacker", 0, 0); drop table users; --')]
    add_students(conn, students)

OperationalError: ignored

In [13]:
with sqlite3.connect('example.db') as conn:
    select_all_students(conn)

OperationalError: ignored

Where is my table, Lebowski???

<img src="https://imgs.xkcd.com/comics/exploits_of_a_mom.png">

[Статья на Хабре про SQL-инъекции](https://habr.com/ru/post/148151/)

# Работа с <имя вашей любимой базы>

Почти ничем не отличается от SQLite.*

\* В случае, если поддержан [Python DB API 2.0](https://www.python.org/dev/peps/pep-0249/)

PostgreSQL: [psycopg2](http://initd.org/psycopg/)

![](https://drive.google.com/uc?export=view&id=10mbDgilFN2xBziq2syPuFHuJzhLSubHM)

In [15]:
try:
    import psycopg2
except:
    !pip install psycopg2-binary

In [ ]:
# Поменялся способ подключения. 
# Теперь указываем не файл, а хост, базу, пользователя и пароль.
# Чтобы это работало, должен быть запущен psqld.
# Также нужно создать пользователя с паролем и базу.
with psycopg2.connect(dbname='testdb', user='testuser', 
                      host='localhost', password='testuser') as conn:
    
    # А остальной интерфейс сохранился.
    cur = conn.cursor()
    cur.execute("select 'Hello, world!', 6 * 7")
    print(cur.fetchone())

('Hello, world!', 42)


[Как установить PostgreSQL в Ubuntu и добавить пользователя](https://www.digitalocean.com/community/tutorials/postgresql-ubuntu-16-04-ru#создание-новой-роли)

Создадим таблички

In [ ]:
with psycopg2.connect(dbname='testdb', user='testuser',
                      host='localhost', password='testuser') as conn:
    
    cur = conn.cursor()
    
    # Удалим старые таблицы, если есть.
    cur.execute('DROP TABLE IF EXISTS users')
    cur.execute('DROP TABLE IF EXISTS user_types')
    
    # SQL диалект может немного меняться
    # Используем тип SERIAL вместо свойстав AUTOINCREMENT
    cur.execute('''
        CREATE TABLE user_types (
            id SERIAL PRIMARY KEY,
            name VARCHAR(255)
        )
    ''')
    cur.execute('''
        CREATE TABLE users (
            id SERIAL PRIMARY KEY,
            first_name VARCHAR(255),
            last_name VARCHAR(255),
            active BOOLEAN,
            profile INTEGER,
            FOREIGN KEY(profile) REFERENCES user_types(id)
        )
    ''')

Многие запросы работают без изменений.

In [ ]:
with psycopg2.connect(dbname='testdb', user='testuser',
                      host='localhost', password='testuser') as conn:
    cur = conn.cursor()
    
    add_user_types(cur)
    add_teachers(cur)
    add_students(cur)
    conn.commit()
    
    select_all_students(conn)

Ivan Ivanov
Petr Petrov
Petr Sidorov


# ORM

**ORM (Object-Relational Mapping)** — технология программирования, которая
связывает базы данных с концепциями объектно-ориентированных языков
программирования, создавая «виртуальную объектную базу данных». Существует
множество вариантов реализации этой технологии.

## Задача

Необходимо обеспечить работу с данными в терминах классов, а не таблиц данных и напротив, преобразовать термины и данные классов в данные, пригодные для хранения в СУБД. Необходимо также обеспечить интерфейс для CRUD-операций над данными. В общем, необходимо избавиться от необходимости писать SQL-код для взаимодействия в СУБД.

## Решение

Решение **проблемы хранения** существует — это реляционные СУБД. Но их использование для хранения объектно-ориентированных данных приводит к семантическому разрыву, заставляя программистов писать программное обеспечение, которое должно уметь как обрабатывать данные в объектно- ориентированном виде, так и уметь сохранить эти данные в реляционной форме.

Разработано множество пакетов, устраняющих необходимость в преобразовании объектов для хранения в реляционных базах данных.
С точки зрения программиста система должна выглядеть как постоянное хранилище объектов. Он может просто создавать объекты и работать с ними как обычно, а они автоматически будут сохраняться в реляционной базе данных.

## Peewee

http://docs.peewee-orm.com/en/latest/

In [ ]:
try:
    from peewee import *
except:
    !pip install peewee
    from peewee import *

db = PostgresqlDatabase(database='postgres', user='merkureva',
                        password='1234', host='localhost')

In [ ]:
class Person(Model):
    name = CharField()
    birthday = DateField()
    is_relative = BooleanField()
    
    # другие типы полей:   https://peewee.readthedocs.io/en/latest/peewee/models.html#field-types-table
    
    class Meta:
        database = db # модель будет использовать базу данных 'postgres'

Инициализирующие аргументы:
- *null=False* – возможно ли хранение null-значений;
- *index=False* – создавать ли индекс для данного столбца в базе;
- *unique=False* – создавать ли уникальный индекс для данного столбца в базе.
- *verbose_name=None* – строка для человекопонятного представления поля;
- *help_text=None* – строка с вспомогательным текстом для поля;
- *db_column=None* – строка, явно задающая название столбца в базе для данного поля, используется например при работе с legacy-базой данных;
- *default=None* – значение по умолчанию для полей класса при инстанцировании;
- *choices=None* – список или кортеж двухэлементных кортежей, где первый элемент – значение для базы, второй – отображаемое значение (аналогично джанге);
- *primary_key=False* – использовать ли данное поле, как первичный ключ;

In [ ]:
class Pet(Model):
    owner = ForeignKeyField(Person, backref='pets')
    name = CharField()
    animal_type = CharField()
    
    class Meta:
        database = db # модель будет использовать базу данных 'postgres'

In [ ]:
# Создаем таблицы в базе со всеми нужными колонками,
# ключами и сиквенсами (если они нужны)
Person.create_table()
Pet.create_table()

In [ ]:
from datetime import date


uncle_bob = Person(name='Bob', birthday=date(1960, 1, 15),
                    is_relative=True)
uncle_bob.save() # cохраним Боба в базе данных

grandma = Person.create(name='Grandma', birthday=date(1935, 3, 1),
                    is_relative=True)
herb = Person.create(name='Herb', birthday=date(1950, 5, 5),
                    is_relative=False)

grandma.name = 'Grandma L.'

grandma.save() # обновим запись grandma

1

In [ ]:
bob_kitty = Pet.create(owner=uncle_bob, name='Kitty', animal_type='cat')
herb_fido = Pet.create(owner=herb, name='Fido', animal_type='dog')
herb_mittens = Pet.create(owner=herb, name='Mittens', animal_type='cat')
herb_mittens_jr = Pet.create(owner=herb, name='Mittens Jr', animal_type='cat')

In [ ]:
grandma = Person.select().where(Person.name == 'Grandma L.').get()
grandma.name

'Grandma L.'

In [ ]:
grandma = Person.get(Person.name == 'Grandma L.')
grandma.name

'Grandma L.'

In [ ]:
for person in Person.select():
    print(person.name, person.is_relative)

Bob True
Herb False
Grandma L. True


In [ ]:
for person in Person.select():
    print(person.name, person.pets.count(), 'pets')
    for pet in person.pets:
        print (' ', pet.name, pet.animal_type)

Bob 1 pets
  Kitty cat
Herb 3 pets
  Fido dog
  Mittens cat
  Mittens Jr cat
Grandma L. 0 pets


In [ ]:
herb_mittens.delete_instance()
herb_fido.owner = uncle_bob
herb_fido.save()

1

In [ ]:
# агрегация значений
for person in Person.select():
    print(person.name, person.pets.count(), 'pets')
    for pet in person.pets:
        print (' ', pet.name, pet.animal_type)

Bob 2 pets
  Kitty cat
  Fido dog
Herb 1 pets
  Mittens Jr cat
Grandma L. 0 pets


In [ ]:
query = Pet.select().where(Pet.animal_type == 'cat')
for pet in query:
    print(pet.name, pet.owner.name)

Kitty Bob
Mittens Jr Herb


In [ ]:
query = (Pet
         .select(Pet, Person)
         .join(Person)
         .where(Pet.animal_type == 'cat'))

for pet in query:
    print(pet.name, pet.owner.name)

Kitty Bob
Mittens Jr Herb


In [ ]:
for pet in Pet.select().join(Person).where(Person.name == 'Bob'):
    print(pet.name)

Kitty
Fido


In [ ]:
for person in Person.select().order_by(Person.birthday.desc()):
    print(person.name)

Bob
Herb
Grandma L.


In [ ]:
# Используем сложное условие в запросе
d1940 = date(1940, 1, 1)
d1960 = date(1960, 1, 1)
query = (Person
         .select()
         .where((Person.birthday < d1940) | (Person.birthday > d1960)))

for person in query:
    print(person.name, person.birthday)

Bob 1960-01-15
Grandma L. 1935-03-01


In [ ]:
query = (Person
         .select()
         .where(Person.birthday.between(d1940, d1960)))

for person in query:
    print(person.name, person.birthday)

Herb 1950-05-05


In [ ]:
# соединение нескольких таблиц
query = (Person
         .select(Person, fn.COUNT(Pet.id).alias('pet_count'))
         .join(Pet, JOIN.LEFT_OUTER)  # не исключаем людей без животных
         .group_by(Person)
         .order_by(Person.name))

for person in query:
    print(person.name, person.pet_count, 'pets')

Bob 2 pets
Grandma L. 0 pets
Herb 1 pets


In [ ]:
db.close()

True

## [SQLAlchemy](https://www.sqlalchemy.org/)

![](https://drive.google.com/uc?export=view&id=1g55PgKhzPXljV4uMAYpDPXjhYHpdw9tr)

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
engine = create_engine('sqlite:///example_alchemy.db', echo=True)
# engine = create_engine('sqlite:///:memory:', echo=True)

# echo=True - для дополнительного логгирования

#### Опишем данные, которые хотим хранить

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Boolean, ForeignKey
from sqlalchemy.orm import relationship

Base = declarative_base()

class UserType(Base):
    __tablename__ = 'user_types'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    users = relationship("User")
    
    def __repr__(self):
        return "<UserType(name={})>".format(self.name)  

class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    first_name = Column(String)
    last_name = Column(String)
    active = Column(Boolean)
    profile_id = Column(Integer, ForeignKey('user_types.id'))
    profile = relationship("UserType", back_populates="users")
    
    def __repr__(self):
        return "<User(first_name={}, last_name={}, active={})>"\
            .format(self.first_name, self.last_name, self.active)
    

In [ ]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('first_name', String(), table=<users>), Column('last_name', String(), table=<users>), Column('active', Boolean(), table=<users>), Column('profile_id', Integer(), ForeignKey('user_types.id'), table=<users>), schema=None)

#### Создадим таблицы в базе

In [ ]:
Base.metadata.create_all(engine)

2019-04-07 00:51:55,800 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-04-07 00:51:55,801 INFO sqlalchemy.engine.base.Engine ()
2019-04-07 00:51:55,802 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-04-07 00:51:55,802 INFO sqlalchemy.engine.base.Engine ()
2019-04-07 00:51:55,803 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("user_types")
2019-04-07 00:51:55,803 INFO sqlalchemy.engine.base.Engine ()
2019-04-07 00:51:55,804 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-04-07 00:51:55,804 INFO sqlalchemy.engine.base.Engine ()
2019-04-07 00:51:55,805 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE user_types (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (id)
)


2019-04-07 00:51:55,805 INFO sqlalchemy.engine.base.Engine ()
2019-04-07 00:51:55,811 INFO sqlalchemy.engine.base.Engine COMMIT
2019-04-07 00:51:55,813 INFO sqlalchemy.engine.base.Engine 
CR

Вставим данные

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

teacher = UserType(name='teacher')
session.add(teacher)

student = UserType(name='student')
session.add(student)

session.commit()

2019-04-07 00:51:56,986 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-07 00:51:56,987 INFO sqlalchemy.engine.base.Engine INSERT INTO user_types (name) VALUES (?)
2019-04-07 00:51:56,987 INFO sqlalchemy.engine.base.Engine ('teacher',)
2019-04-07 00:51:56,989 INFO sqlalchemy.engine.base.Engine INSERT INTO user_types (name) VALUES (?)
2019-04-07 00:51:56,989 INFO sqlalchemy.engine.base.Engine ('student',)
2019-04-07 00:51:56,990 INFO sqlalchemy.engine.base.Engine COMMIT


Найдем тип учителя в базе

In [ ]:
res = session.query(UserType).filter_by(name='teacher').first() 
res

2019-04-07 00:52:15,470 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-07 00:52:15,471 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.name = ?
 LIMIT ? OFFSET ?
2019-04-07 00:52:15,471 INFO sqlalchemy.engine.base.Engine ('teacher', 1, 0)


<UserType(name=teacher)>

In [ ]:
teacher is res

True

one vs first

In [ ]:
session.query(UserType).filter_by(name='teacher12').one() 

2019-04-07 00:52:17,544 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.name = ?
2019-04-07 00:52:17,545 INFO sqlalchemy.engine.base.Engine ('teacher12',)


NoResultFound: No row was found for one()

In [ ]:
session.query(UserType).one() 

2019-04-07 00:52:20,111 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types
2019-04-07 00:52:20,112 INFO sqlalchemy.engine.base.Engine ()


MultipleResultsFound: Multiple rows were found for one()

Разные виды "select"

In [ ]:
results = session.query(UserType).order_by(UserType.id).all()
print(type(results))
print(results)

2019-04-07 00:52:21,436 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types ORDER BY user_types.id
2019-04-07 00:52:21,438 INFO sqlalchemy.engine.base.Engine ()
<class 'list'>
[<UserType(name=teacher)>, <UserType(name=student)>]


In [ ]:
for name, user_id in session.query(UserType.name, UserType.id).order_by(UserType.name):
    print(name, user_id)

2019-04-07 00:52:21,959 INFO sqlalchemy.engine.base.Engine SELECT user_types.name AS user_types_name, user_types.id AS user_types_id 
FROM user_types ORDER BY user_types.name
2019-04-07 00:52:21,960 INFO sqlalchemy.engine.base.Engine ()
student 2
teacher 1


#### Добавим нового учителя

In [ ]:
user = User(first_name="Maxim", last_name="Popov", active=True)
teacher.users.append(user)
session.add(user)
session.commit()

2019-04-07 00:52:23,111 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users 
WHERE ? = users.profile_id
2019-04-07 00:52:23,113 INFO sqlalchemy.engine.base.Engine (1,)
2019-04-07 00:52:23,120 INFO sqlalchemy.engine.base.Engine INSERT INTO users (first_name, last_name, active, profile_id) VALUES (?, ?, ?, ?)
2019-04-07 00:52:23,121 INFO sqlalchemy.engine.base.Engine ('Maxim', 'Popov', 1, 1)
2019-04-07 00:52:23,124 INFO sqlalchemy.engine.base.Engine COMMIT


In [ ]:
res = session.query(User).first() 
res

2019-04-07 00:52:23,662 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-07 00:52:23,664 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users
 LIMIT ? OFFSET ?
2019-04-07 00:52:23,664 INFO sqlalchemy.engine.base.Engine (1, 0)


<User(first_name=Maxim, last_name=Popov, active=True)>

In [ ]:
res.profile

2019-04-07 00:52:24,221 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2019-04-07 00:52:24,223 INFO sqlalchemy.engine.base.Engine (1,)


<UserType(name=teacher)>

Добавим студента

In [ ]:
user = User(first_name="Petr", last_name="Sidorov", active=False, profile=student)
session.add(user)
session.commit()

2019-04-07 00:52:25,274 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2019-04-07 00:52:25,275 INFO sqlalchemy.engine.base.Engine (2,)
2019-04-07 00:52:25,276 INFO sqlalchemy.engine.base.Engine INSERT INTO users (first_name, last_name, active, profile_id) VALUES (?, ?, ?, ?)
2019-04-07 00:52:25,277 INFO sqlalchemy.engine.base.Engine ('Petr', 'Sidorov', 0, 2)
2019-04-07 00:52:25,278 INFO sqlalchemy.engine.base.Engine COMMIT


In [ ]:
student.users

2019-04-07 00:52:25,799 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-07 00:52:25,799 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2019-04-07 00:52:25,800 INFO sqlalchemy.engine.base.Engine (2,)
2019-04-07 00:52:25,801 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users 
WHERE ? = users.profile_id
2019-04-07 00:52:25,801 INFO sqlalchemy.engine.base.Engine (2,)


[<User(first_name=Petr, last_name=Sidorov, active=False)>]

Сделаем его учителем

In [ ]:
user.profile = teacher
session.commit()

2019-04-07 00:52:29,822 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2019-04-07 00:52:29,825 INFO sqlalchemy.engine.base.Engine (1,)
2019-04-07 00:52:29,829 INFO sqlalchemy.engine.base.Engine UPDATE users SET profile_id=? WHERE users.id = ?
2019-04-07 00:52:29,832 INFO sqlalchemy.engine.base.Engine (1, 2)
2019-04-07 00:52:29,836 INFO sqlalchemy.engine.base.Engine COMMIT


In [ ]:
student.users

2019-04-07 00:52:30,582 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-07 00:52:30,584 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2019-04-07 00:52:30,586 INFO sqlalchemy.engine.base.Engine (2,)
2019-04-07 00:52:30,589 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users 
WHERE ? = users.profile_id
2019-04-07 00:52:30,590 INFO sqlalchemy.engine.base.Engine (2,)


[]

In [ ]:
teacher.users

2019-04-07 00:52:31,312 INFO sqlalchemy.engine.base.Engine SELECT user_types.id AS user_types_id, user_types.name AS user_types_name 
FROM user_types 
WHERE user_types.id = ?
2019-04-07 00:52:31,313 INFO sqlalchemy.engine.base.Engine (1,)
2019-04-07 00:52:31,315 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users 
WHERE ? = users.profile_id
2019-04-07 00:52:31,316 INFO sqlalchemy.engine.base.Engine (1,)


[<User(first_name=Maxim, last_name=Popov, active=True)>,
 <User(first_name=Petr, last_name=Sidorov, active=False)>]

Какая еще бывает фильтрация?

In [ ]:
# equals
session.query(UserType).filter(UserType.name == 'teacher')
# not equals
session.query(UserType).filter(UserType.name != 'teacher')
# LIKE
session.query(UserType).filter(UserType.name.like('%tea%'))
# ILIKE (case-insensitive LIKE):
session.query(UserType).filter(UserType.name.ilike('%TeA%'))
# IS NULL
session.query(UserType).filter(UserType.name.is_(None))
# IS NOT NULL
session.query(UserType).filter(UserType.name.isnot(None))
              
from sqlalchemy import and_
#AND
session.query(UserType).filter(and_(UserType.name.isnot(None), UserType.name != 'abc'))
session.query(UserType).filter(UserType.name.isnot(None), UserType.name != 'abc')
session.query(UserType).filter(UserType.name.isnot(None)).filter(UserType.name != 'abc')

#OR
from sqlalchemy import or_
session.query(UserType).filter(or_(UserType.name.isnot(None), UserType.name != 'abc'))

In [ ]:
#IN
session.query(User).filter(User.profile_id.in_(
    session.query(UserType.id)
)).all()

2019-04-07 00:52:33,498 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.first_name AS users_first_name, users.last_name AS users_last_name, users.active AS users_active, users.profile_id AS users_profile_id 
FROM users 
WHERE users.profile_id IN (SELECT user_types.id AS user_types_id 
FROM user_types)
2019-04-07 00:52:33,499 INFO sqlalchemy.engine.base.Engine ()


[<User(first_name=Maxim, last_name=Popov, active=True)>,
 <User(first_name=Petr, last_name=Sidorov, active=False)>]

## ORM: плюсы и минусы

ORM, по идее, должен избавить нас от написания SQL запросов и, в идеале, вообще абстрагировать от базы данных (от способа хранения данных), чтобы мы могли работать с классами, в той или иной степени выражающими объекты бизнес-логики, не задаваясь вопросом, в каких таблицах всё это по факту лежит.

Для примера возьмем две таблицы: книги и авторы книг, отношение многие-ко- многим (у книг может быть много авторов, у авторов может быть много книг). Т.е. в базе это будут books, authors и связующая таблица author_book:

```sql
 CREATE TABLE authors (
    id serial
  , name varchar(1000) not null
  , PRIMARY KEY (id)
);
CREATE TABLE books (
    id serial
  , name VARCHAR (1000) not null
  , text text not null
  , PRIMARY KEY (id)
);
CREATE TABLE author_book (
    author_id bigint REFERENCES authors(id)
  , book_id bigint REFERENCES books(id)
  , PRIMARY KEY (author_id, book_id)
);
```

## ORM: плюсы и минусы

### Кейс 1: создание записей

- Pure SQL
    - Piece of cake. Простые CREATE и INSERT
    - Много писанины
    - Нужно знать синтаксис SQL
- ORM
    - Piece of cake. Создаём нужные классы, наполняем
    - Много писанины
    - Нужно разобраться с ORM

### Кейс 2: обновление записей

- Pure SQL
    - Piece of cake. Просто UPDATE
- ORM
    - Piece of cake. Пользуемся нужным методом (как с именем бабули ранее)

### Кейс 3: простая выборка
-  Pure SQL
    - Piece of cake. Запрос с агрегацией:
    ```sql
      select
        b.id as book_id
        , b.name as book_name
        , json_agg(a.name) as authors
      from
        books b
      inner join
        author_book ab
          on b.id = ab.book_id
      inner join
        authors a
          on ab.author_id = a.id
      group by
        b.id;
```
- ORM
    - Piece of cake (так ли это?)
        - Проходим циклом и находим нужное
        - Будет несколько запросов: выборка всех книг и для каждой выборка автора
        - С ростом количества данных будет выполняться всё дольше
        - Вытянутся все поля, а не только нужные; придётся разбираться с SQL

## ORM: плюсы и минусы

### Кейс 4: сложное обновление

- Pure SQL
    - Piece of cake. Простой подзапрос
    ```sql
      UPDATE authors
      SET name = 'Лев Толстой'
      WHERE id in (
          SELECT
            id
          FROM
            authors
          ORDER BY
            id DESC
          LIMIT 2 );
    ```
- ORM
    - Долгое изучение мануалов
    - Никаких подзапросов
    - Неоптимально с точки зрения БД